In [8]:
from tqdm import tqdm

## 1. Pre-train on CIFAR-10

In [ ]:
'''Train CIFAR10 with PyTorch.'''
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from models import *
from utils import progress_bar

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[150, 250], gamma=0.1)

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    with tqdm(total=len(trainloader)) as pbar:
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
            pbar.update(1)

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with tqdm(total=len(testloader)) as pbar:
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(testloader):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = net(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

                pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
                pbar.update(1)

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('/tmp/work/checkpoint'):
            os.mkdir('/tmp/work/checkpoint')
        torch.save(state, '/tmp/work/checkpoint/ckpt.cifar10.t7')
        best_acc = acc


for epoch in range(start_epoch, 350):
    scheduler.step()
    train(epoch)
    if epoch % 10 == 0:
        test(epoch)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/391 [00:00<?, ?it/s]

==> Building model..

Epoch: 0


  0%|          | 0/391 [00:00<?, ?it/s]

Saving..

Epoch: 1


...

  0%|          | 0/391 [00:00<?, ?it/s]

Saving..

Epoch: 5


  0%|          | 0/391 [00:00<?, ?it/s]


Epoch: 6


Acc: 76.374%:  91%|█████████ | 355/391 [00:29<00:02, 12.14it/s]

testing accuracy: >90%

## Train on CINIC-10

- train: train set
- valid: valid set

In [ ]:
'''Train CINIC10 with PyTorch.'''
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from models import *
from utils import progress_bar

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
])

trainset = torchvision.datasets.ImageFolder(root='/tmp/work/data/CINIC-10/train/', transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.ImageFolder(root='/tmp/work/data/CINIC-10/valid/', transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

# Load checkpoint.
print('==> Resuming from checkpoint..')
checkpoint = torch.load('/tmp/work/checkpoint/ckpt.cifar10.t7')
net.load_state_dict(checkpoint['net'])
#start_epoch = checkpoint['epoch']
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[150, 250], gamma=0.1)

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx, len(trainloader), 'Acc: %.3f%%'
            % (100.*correct/total))

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(testloader), 'Acc: %.3f%%'
                % (100.*correct/total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('/tmp/work/checkpoint'):
            os.mkdir('/tmp/work/checkpoint')
        torch.save(state, '/tmp/work/checkpoint/ckpt.cinic10.0.t7')
        best_acc = acc


for epoch in range(start_epoch, 350):
    scheduler.step()
    train(epoch)
    if epoch % 10 == 0:
        test(epoch)

Accuracy 77%

## Train on full CINIC-10

- train: train set + valid set
- valid: test set

In [ ]:
'''Train CINIC10 with PyTorch.'''
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from models import *
from utils import progress_bar

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
])

trainset = torchvision.datasets.ImageFolder(root='/tmp/work/data/CINIC-10/train/', transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

validset = torchvision.datasets.ImageFolder(root='/tmp/work/data/CINIC-10/valid/', transform=transform_train)
validloader = torch.utils.data.DataLoader(validset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.ImageFolder(root='/tmp/work/data/CINIC-10/valid/', transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

# Load checkpoint.
print('==> Resuming from checkpoint..')
checkpoint = torch.load('/tmp/work/checkpoint/ckpt.cinic10.0.t7')
net.load_state_dict(checkpoint['net'])
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[150, 250], gamma=0.1)

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx, len(trainloader), 'Acc: %.3f%%'
            % (100.*correct/total))

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(testloader), 'Acc: %.3f%%'
                % (100.*correct/total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('/tmp/work/checkpoint'):
            os.mkdir('/tmp/work/checkpoint')
        torch.save(state, '/tmp/work/checkpoint/ckpt.cinic10.1.t7')
        best_acc = acc

for epoch in range(start_epoch, 350):
    scheduler.step()
    train(epoch)
    if epoch % 10 == 0:
        test(epoch)

## Train on full CINIC-10 with data augmentation

- train: train set + valid set
- valid: test set

In [ ]:
'''Train CINIC10 with PyTorch.'''
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from models import *
from utils import progress_bar


from imgaug import augmenters as iaa
from imgaug import parameters as iap
import numpy as np
import PIL

class ImgAugTransform:
    def __init__(self):
        self.aug = iaa.Sequential([
            iaa.SomeOf((0, 2), [
                iaa.Affine(rotate=iap.Uniform(0.0, 360.0)),
                iaa.CropAndPad(percent=(-0.2, 0)),
                iaa.Fliplr(0.5),
                iaa.Flipud(0.5),
                iaa.AdditiveGaussianNoise(scale=0.1*255),
                iaa.Sharpen(alpha=0.5),
                iaa.PiecewiseAffine(scale=iap.Uniform(0.02, 0.08), 
                                    nb_rows=iap.Uniform(4,8),
                                    nb_cols=iap.Uniform(4,8))
            ]),
        ])

    def __call__(self, img):
        img = np.array(img)
        return self.aug.augment_image(img)


device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
    ImgAugTransform(),
    lambda x: PIL.Image.fromarray(x),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
])

trainset = torchvision.datasets.ImageFolder(root='/tmp/work/data/CINIC-10/train/', transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

validset = torchvision.datasets.ImageFolder(root='/tmp/work/data/CINIC-10/valid/', transform=transform_train)
validloader = torch.utils.data.DataLoader(validset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.ImageFolder(root='/tmp/work/data/CINIC-10/valid/', transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

# Load checkpoint.
print('==> Resuming from checkpoint..')
checkpoint = torch.load('/tmp/work/checkpoint/ckpt.cinic10.1.t7')
net.load_state_dict(checkpoint['net'])
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[150, 250], gamma=0.1)

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx, len(trainloader), 'Acc: %.3f%%'
            % (100.*correct/total))

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(testloader), 'Acc: %.3f%%'
                % (100.*correct/total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('/tmp/work/checkpoint'):
            os.mkdir('/tmp/work/checkpoint')
        torch.save(state, '/tmp/work/checkpoint/ckpt.cinic10.2.t7')
        best_acc = acc


for epoch in range(start_epoch, 350):
    scheduler.step()
    train(epoch)
    test(epoch)

## Model Deployment

In [1]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable

import os
import argparse

from models import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Model
print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

# Load checkpoint.
print('==> Resuming from checkpoint..')
checkpoint = torch.load('/tmp/work/checkpoint/ckpt.cinic10.0.t7')
net.load_state_dict(checkpoint['net'])

# Export model
print('==> Exporting model..')
dummy_input = Variable(torch.randn(1, 3, 32, 32), requires_grad=False)
torch.onnx.export(net, dummy_input, "../mobileNetV2.cinic10.onnx", verbose=True)

==> Building model..
==> Resuming from checkpoint..


RuntimeError: Error(s) in loading state_dict for DataParallel:
	Unexpected key(s) in state_dict: "module.bn1.num_batches_tracked", "module.layers.0.bn1.num_batches_tracked", "module.layers.0.bn2.num_batches_tracked", "module.layers.0.bn3.num_batches_tracked", "module.layers.0.shortcut.1.num_batches_tracked", "module.layers.1.bn1.num_batches_tracked", "module.layers.1.bn2.num_batches_tracked", "module.layers.1.bn3.num_batches_tracked", "module.layers.1.shortcut.1.num_batches_tracked", "module.layers.2.bn1.num_batches_tracked", "module.layers.2.bn2.num_batches_tracked", "module.layers.2.bn3.num_batches_tracked", "module.layers.3.bn1.num_batches_tracked", "module.layers.3.bn2.num_batches_tracked", "module.layers.3.bn3.num_batches_tracked", "module.layers.4.bn1.num_batches_tracked", "module.layers.4.bn2.num_batches_tracked", "module.layers.4.bn3.num_batches_tracked", "module.layers.5.bn1.num_batches_tracked", "module.layers.5.bn2.num_batches_tracked", "module.layers.5.bn3.num_batches_track